# Function Dependency Equation for Free Product of 2 Cyclic Groups.

Consider $G=\langle x| x^m=1 \rangle * \langle y| y^n=1 \rangle$, where $m,n \in \mathbb{Z}_{\ge0}\setminus \{1\}$. Note that $|G|<\infty \iff mn>0$. 
Take the generating set $S=S_x\cup S_y$ for $G$, where 
$$S_x=\{x\}\cup \iota(m=0)\cdot\{x^{-1}\}, S_y=\{y\}\cup \iota(n=0)\cdot\{y^{-1}\},$$ 
and $$\iota(\mathcal{P})=\begin{cases}\{1\},&\mathcal{P}\\ \emptyset,& \lnot \mathcal{P}\end{cases}$$ with $\cdot$ denoting elementwise group multiplication on sets [$A\cdot B = \{ab: a\in A,\ b\in B\}$]. 

For $g\in G$ and $X\subseteq G$, let $F_{g,X}(t)$ be series for the set of all words in $S^*$ equivalent to $g\in G$, with proper nonempty prefixes avoiding $X$, characterized by the word length. We set up functional equations involving $F_{g,X}$, for which $\{g\}\cup X\subseteq \{v^k: k\in \mathbb{Z}\}$, and $v\in \{x,y\}$. Finally, we solve for $F(t):=F_{1,\emptyset}(t)$.

In [313]:
# choose your desired values for m,n
# for now, only work for finite G, (mn>0)
m = 3
n = 3

# Do we want word concatenation to reduce to group multiplication?
AUTO_REDUCE = True
DEFAULT_RS = None # a rewiting system in order to make group element reduction possible, will be set later


In [314]:
# need to define set-set, set-element multiplication
def set_mult(A,B):
    # turn any non-set object into a singleton set containing that object
    A,B = map(lambda S: S if S in Sets else Set([S]),[A,B])
    C= Set([a*b for a,b in cartesian_product([A,B])]) # assuming element multiplication is well defined
    if AUTO_REDUCE and DEFAULT_RS is not None:
        C = Set([DEFAULT_RS.reduce(c) for c in C])
    return C

# Let's overload the '*' operator for Set objects
# Set is a function, but returns an instance of some spectifed kind of Set class, which is accessible via .parent()
reverse_if = lambda seq, cond: list(seq) if not cond else list(reversed(seq))
Set().parent()._mul_ =  lambda self, other, switch_sides=False: set_mult(*(reverse_if([self,other], switch_sides)))

In [315]:
# compute the free product
F2.<x,y> = FreeGroup()
G = F2/ (x^m,y^n)
x,y = G.gens()

# need to write words in reduced form
Grs = G.rewriting_system()
DEFAULT_RS = Grs

In [316]:
# test it out
A = Set([x,y,G.one()])

print(x*A) # with reduction

# very simple to supress/invoke group reduction
AUTO_REDUCE = False
print(x*A) # without reduction
AUTO_REDUCE = True
print(x*A) # with reduction

{x*y, x^2, x}
{x*y, x^2, x}
{x*y, x^2, x}


In [317]:
# latex expression for sets
def set_to_latex(A):
    if A == Set():
        return r'\emptyset'
    return latex(A)

In [318]:
# define a space of variables indexed by the (g,X) pairs
# start with what we wish to solve, (1,\emptyset).
def series_expression(g,X):
    return 'F_{%s,%s}(t)'%(latex(g),set_to_latex(X))
var_space = {}
goal_pair = (G.one(), Set())
VAR_COUNTER = 0 # increment every time a new series variable is defined

def add_series_var(pair):
    if pair in var_space:
        return False
    global VAR_COUNTER
    g,X = pair
    var_space[pair] = var('v%d'%VAR_COUNTER, latex_name = series_expression(g,X))
    VAR_COUNTER += 1
    return True

add_series_var(goal_pair)
    

True

In [319]:
pretty_print(var_space)

{(1, {}): v0}

In [320]:
# Now get the cyclic factors

# Sage is not good with subgroups
# Gx = G.subgroup((x,))
# Gy = G.subgroup((y,))
# xx,yy = Gx.gens()+Gy.gens()

def in_cyclic_factor(el, v):
    '''
        returns if the group element is in the given cyclic factor
        el: the element in G to check
        v: either x or y, the generator of the factor 
    '''
    expr = el.syllables()
    if not expr:
        return True
    return len(expr)==1 and expr[0][0] == v
    
def subset_of_cyclic_factor(A,v):
    # assume finite sets only
    return all((in_cyclic_factor(el,v) for el in A))

In [321]:
# for finite cyclic factors, we don't want negative exponents. We redefine the __pow__ method.
if AUTO_REDUCE and DEFAULT_RS is not None:
    if '_old_pow' not in vars():
        _old_pow = type(x).__pow__
    def _new_pow(a, k):
        if k not in ZZ or k>=0:
            # case isn't broken, don't change it
            return _old_pow(a,k)

        # k is a negative integer
        ans = _old_pow(a,k)
    #     if AUTO_REDUCE:
    #         ans = DEFAULT_RS.reduce(ans)
        nans = G.one()
        for v,ep in ans.syllables():
            modval = m if v==x else n
            nans*= _old_pow(v, ep if modval==0 else ep%modval)
        return nans
    type(x).__pow__ = _new_pow

In [322]:
# For multiplication, want auto reduation to be available
# if AUTO_REDUCE and DEFAULT_RS is not None:
#     if '_old_mul' not in vars():
#         _old_mul = type(x).__mul__
#     def _new_mul(a, b):
#         return DEFAULT_RS.reduce(_old_mul(a,b))
#     type(x).__mul__ = _new_mul


In [323]:
x,x^2,x^3*x,x^-1,(x*y)^-1, x^-2,y^-3,(x*y^3)^-1, '_old_pow' in vars()

(x, x^2, x^4, x^2, y^2*x^2, x, 1, x^2, True)

In [324]:
var('t')

t

In [325]:
# construct the functional equation given the desired (g,X) pair
# assume {g}\cup X is completely in one of the cyclic factors
# CURRENTLY FINITE CASE ONLY

def get_FD_equation(pair, ret_new_pairs = True):
    '''
        returns a functional depency equation
        if ret_new_pairs is set to True, return a (equation, set of (g,X) pairs) pair instead
    '''
    if ret_new_pairs:
        newpairs = set()
    g,X = pair
    eqn = None
    one = G.one()
    oneset = Set([one])
    if one not in X:
        XU1 = X.union(oneset)
        if add_series_var((one,X)) and ret_new_pairs:
            newpairs.add((one,X))
        if add_series_var((g,XU1)) and ret_new_pairs:
            newpairs.add((g,XU1))
        if g==one:
            eqn = 1+var_space[(one,X)]*(var_space[(one,XU1)]-1)
        else:
            eqn = var_space[(one,X)]*var_space[(g,XU1)]
    elif subset_of_cyclic_factor(X,x):
        if g!=one:
            eqn = t*ZZ(g==x)
            if x not in X:
                [g] = x^-1*Set([g]) # use set so auto reduction is implemented
                X = x^-1*X
                if add_series_var((g,X)) and ret_new_pairs:
                    newpairs.add((g,X))
                eqn += t*var_space[(g,X)]
        else:
            set_yinv = Set([y^-1]) 
            if add_series_var((y^-1,set_yinv)) and ret_new_pairs:
                newpairs.add((y^-1,set_yinv))
            eqn = 1+t*var_space[(y^-1,set_yinv)]
            if x not in X:
                g = x^-1
                X = x^-1*X
                if add_series_var((g,X)) and ret_new_pairs:
                    newpairs.add((g,X))
                eqn += t*var_space[(g,X)]
    else:
        # the y factor case
        if g!=one:
            eqn = t*ZZ(g==y)
            if y not in X:
                [g] = y^-1*Set([g])
                X = y^-1*X
                if add_series_var((g,X)) and ret_new_pairs:
                    newpairs.add((g,X))
                eqn += t*var_space[(g,X)]
        else:
            set_xinv = Set([x^-1]) 
            if add_series_var((x^-1,set_xinv)) and ret_new_pairs:
                newpairs.add((x^-1,set_xinv))
            eqn = 1+t*var_space[(x^-1,set_xinv)]
            if y not in X:
                g = y^-1
                X = y^-1*X
                if add_series_var((g,X)) and ret_new_pairs:
                    newpairs.add((g,X))
                eqn += t*var_space[(g,X)]
    eqn = var_space[pair]==eqn
    return (eqn,newpairs) if ret_new_pairs else eqn
    


In [326]:
# Let us build the system of equations
system = []
queue = [goal_pair]

# use BFS
while queue:
    pair = queue.pop(0)
    eqn, newp = get_FD_equation(pair)
    queue.extend(newp)
    system.append(eqn)


In [327]:
# Here are the equations
for eqn in system:
    print(latex(eqn),r'\\')
print()
print()
for eqn in system:
    #print(eqn)
    pretty_print(eqn)

{F_{1,\emptyset}(t)} = {F_{1,\emptyset}(t)} {\left({F_{1,\left\{1\right\}}(t)} - 1\right)} + 1 \\
{F_{1,\left\{1\right\}}(t)} = t {F_{y^{2},\left\{y^{2}\right\}}(t)} + t {F_{x^{2},\left\{x^{2}\right\}}(t)} + 1 \\
{F_{y^{2},\left\{y^{2}\right\}}(t)} = {F_{1,\left\{y^{2}\right\}}(t)} {F_{y^{2},\left\{1, y^{2}\right\}}(t)} \\
{F_{x^{2},\left\{x^{2}\right\}}(t)} = {F_{1,\left\{x^{2}\right\}}(t)} {F_{x^{2},\left\{1, x^{2}\right\}}(t)} \\
{F_{1,\left\{y^{2}\right\}}(t)} = {F_{1,\left\{y^{2}\right\}}(t)} {\left({F_{1,\left\{1, y^{2}\right\}}(t)} - 1\right)} + 1 \\
{F_{y^{2},\left\{1, y^{2}\right\}}(t)} = t {F_{y,\left\{y, y^{2}\right\}}(t)} \\
{F_{x^{2},\left\{1, x^{2}\right\}}(t)} = t {F_{x,\left\{x^{2}, x\right\}}(t)} \\
{F_{1,\left\{x^{2}\right\}}(t)} = {\left({F_{1,\left\{1, x^{2}\right\}}(t)} - 1\right)} {F_{1,\left\{x^{2}\right\}}(t)} + 1 \\
{F_{1,\left\{1, y^{2}\right\}}(t)} = t {F_{y^{2},\left\{y, y^{2}\right\}}(t)} + t {F_{x^{2},\left\{x^{2}\right\}}(t)} + 1 \\
{F_{y,\left\{y, y^{2}\

v0 == v0*(v1 - 1) + 1

v1 == t*v2 + t*v3 + 1

v2 == v4*v5

v3 == v6*v7

...

v17 == v15*v21

v18 == 0

v19 == t*v3 + 1

v20 == t*v2 + 1

v21 == 0

In [328]:
# Finally solve the system

# reports an empty list
# solve(system, *var_space.values(), solution_dict=True)
# X=Set([G.one(),y])
# pretty_print(get_FD_equation((G.one(),X),False))
# subset_of_cyclic_factor(X,y)

In [329]:
# first, isolate variables on the LHS

for i, eqn in enumerate(system):
    system[i] = solve(eqn,eqn.lhs())[0]
    

In [330]:
for eqn in system:
    pretty_print(eqn)

v0 == -1/(v1 - 2)

v1 == t*v2 + t*v3 + 1

v2 == v4*v5

v3 == v6*v7

v4 == -1/(v8 - 2)

...

v17 == v15*v21

v18 == 0

v19 == t*v3 + 1

v20 == t*v2 + 1

v21 == 0

In [331]:
# now make some substitutions

change = 1
numit = 0

while change:
    change = 0
    # build a dependency graph
    eq_by_lhs = {eq.lhs():eq for eq in system}
    dep_gr = {eq.lhs():set(eq.rhs().args())-{t} for eq in system}
    vars_det = [v for v,args in dep_gr.items() if not args] # the completely solved vars

    rev_dep_gr = {v:set() for v in eq_by_lhs} # diagraph by reversing the arcs of dep_gr
    for v,ag in dep_gr.items():
        for w in ag:
            rev_dep_gr[w].add(v)

    # apply BFS to make the substitutions possible
    vis = set(vars_det)
    queue = list(vis)
    done = set()
    while queue:
        nxtvar = queue.pop(0)
        # check if all RHS variables are visited before
        sub_dict = {}
        for dps in dep_gr[nxtvar]:
            if dps not in done:
                continue # can't substitute, a RHS varible is not yet reduced
            sub_dict[dps] = eq_by_lhs[dps].rhs()
            change = 1
        eq_by_lhs[nxtvar] = eq_by_lhs[nxtvar].subs(sub_dict)
        eq_by_lhs[nxtvar].expand()
        for pt in rev_dep_gr[nxtvar]:
            if pt not in vis:
                queue.append(pt)
                vis.add(pt)
        done.add(nxtvar)
    
    for i, eq in enumerate(system):
        system[i] = eq_by_lhs[eq.lhs()]
    numit+=1

print('Number of iteration taken to substitute',numit)
        


Number of iteration taken to substitute 2


In [332]:
for eq in eq_by_lhs.values():
    pretty_print(eq)

v0 == -1/(t^3*v15*v6 - t^3*v13/(t*v3 - 1) - 1)

v1 == t^3*v15*v6 - t^3*v13/(t*v3 - 1) + 1

v2 == -t^2*v13/(t*v3 - 1)

v3 == t^2*v15*v6

v4 == -1/(t*v3 - 1)

...

v17 == 0

v18 == 0

v19 == t^3*v15*v6 + 1

v20 == -t^3*v13/(t*v3 - 1) + 1

v21 == 0

In [333]:
# get rid of every irrelevant variable
start = var_space[goal_pair]
eq_by_lhs = {eq.lhs():eq for eq in system}
dep_gr = {eq.lhs():set(eq.rhs().args())-{t} for eq in system}

comp_set = {start}
comp = [start]
ind = 0
while ind<len(comp):
    for v in dep_gr[comp[ind]]:
        if v not in comp_set:
            comp.append(v)
            comp_set.add(v)
    ind += 1

system = [eq for eq in system if eq.lhs() in comp_set]

In [334]:
pretty_print(comp)
for eq in system:
    print(eq)
    pretty_print(eq)

[v0, v6, v3, v15, v13, v2]

v0 == -1/(t^3*v15*v6 - t^3*v13/(t*v3 - 1) - 1)


v0 == -1/(t^3*v15*v6 - t^3*v13/(t*v3 - 1) - 1)

v2 == -t^2*v13/(t*v3 - 1)


v2 == -t^2*v13/(t*v3 - 1)

...

v6 == -1/(t*v2 - 1)

v13 == -1/(t^3*v15*v6 - 1)


v13 == -1/(t^3*v15*v6 - 1)

v15 == (1/(t^3*v13/(t*v3 - 1) + 1))


v15 == (1/(t^3*v13/(t*v3 - 1) + 1))

In [335]:
# make every equation eplicit. delete loops
# maybe not a good idea due to the posssible interference of radicals

# for i, eqn in enumerate(system):
#     system[i] = solve(eqn,eqn.lhs())[0]

In [336]:
for eq in system:
    print(eq)
    pretty_print(eq)

v0 == -1/(t^3*v15*v6 - t^3*v13/(t*v3 - 1) - 1)


v0 == -1/(t^3*v15*v6 - t^3*v13/(t*v3 - 1) - 1)

v2 == -t^2*v13/(t*v3 - 1)


v2 == -t^2*v13/(t*v3 - 1)

v3 == t^2*v15*v6


...

v6 == -1/(t*v2 - 1)

v13 == -1/(t^3*v15*v6 - 1)


v13 == -1/(t^3*v15*v6 - 1)

v15 == (1/(t^3*v13/(t*v3 - 1) + 1))


v15 == (1/(t^3*v13/(t*v3 - 1) + 1))

In [337]:
# order the component based on the shortest distance from the start in decreasing order
# BFS already put them in increasing order
comp.reverse()
comp.pop(); # get rid of start; the only variable we don't want to eliminate
comp

[v2, v13, v15, v3, v6]

In [338]:
# Expression involving t and F(t) to set = 0
possible_impl_solns = []

for comp_perm in Permutations(comp):
    comp_perm = list(comp_perm)
    print(comp_perm,type(comp_perm))
    possible_impl_solns_perm = maxima.eliminate(system, comp_perm)
    possible_impl_solns_perm = ([sln.sage() for sln in possible_impl_solns_perm])
    possible_impl_solns.extend(possible_impl_solns_perm)
    for impl_soln in possible_impl_solns_perm:
        pretty_print(impl_soln)
        if impl_soln!=0:
            break # found an actual solution
    else:
        continue
    break
    
#solve(system, comp, algorithm='sympy')
#impl_soln(v0=sqrt(1/(1-4*t^2))).simplify_full()

[v2, v13, v15, v3, v6] <class 'list'>


0

0

[v2, v13, v15, v6, v3] <class 'list'>


0

...

[v6, v3, v15, v2, v13] <class 'list'>


0

[v6, v3, v15, v13, v2] <class 'list'>


0

0

In [312]:
# Solve if we can
expl_slns = solve(impl_soln, start, solution_dict=True)

for sol in expl_slns:
    F = sol[start]
    L = F.limit(t=0)
    pretty_print(LatexExpr(r'\lim_{t\to 0}%s=%d'%(latex(start),L)))
    if L!=1:
        print('limit at t=0 is different from 1, discard!')
    else:
        print('possible desired series solutions!')
        #pretty_print(F)
        prec = 70
        pretty_print('Series Expansion around %s of degree up to %d: '%(LatexExpr('t=0'),prec))
        pretty_print(F.taylor(t, 0, prec))


\lim_{t\to 0}{F_{1,\emptyset}(t)}=-1

limit at t=0 is different from 1, discard!


\lim_{t\to 0}{F_{1,\emptyset}(t)}=1

possible desired series solutions!


'Series Expansion around t=0 of degree up to 70: '

112186277816662845432*t^70 + 28453041475240576740*t^68 + 7219428434016265740*t^66 + 1832624140942590534*t^64 + 465428353255261088*t^62 + 118264581564861424*t^60 + 30067266499541040*t^58 + 7648690600760440*t^56 + 1946939425648112*t^54 + 495918532948104*t^52 + 126410606437752*t^50 + 32247603683100*t^48 + 8233430727600*t^46 + 2104098963720*t^44 + 538257874440*t^42 + 137846528820*t^40 + 35345263800*t^38 + 9075135300*t^36 + 2333606220*t^34 + 601080390*t^32 + 155117520*t^30 + 40116600*t^28 + 10400600*t^26 + 2704156*t^24 + 705432*t^22 + 184756*t^20 + 48620*t^18 + 12870*t^16 + 3432*t^14 + 924*t^12 + 252*t^10 + 70*t^8 + 20*t^6 + 6*t^4 + 2*t^2 + 1

\lim_{t\to 0}{F_{1,\emptyset}(t)}=0

limit at t=0 is different from 1, discard!
